## 1. Imports and Configuration

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import json
from pathlib import Path
import sys

# Import custom modules
from edge_matching import (
    PuzzleEdge, extract_piece_edges, compute_edge_similarity,
    find_edge_matches, load_processed_pieces, extract_all_edges_from_puzzle
)
from puzzle_visualization import (
    visualize_edge_matches, visualize_puzzle_graph,
    visualize_edge_descriptors, visualize_assembly_result,
    save_match_report, load_piece_images
)
from puzzle_assembly import PuzzleSolver, simple_neighbor_matching

%matplotlib inline
plt.rcParams['figure.dpi'] = 100

## 2. Configuration

In [ ]:
# Paths
PROCESSED_DIR = "./processed_artifacts"
OUTPUT_DIR = "./phase2_results"
Path(OUTPUT_DIR).mkdir(exist_ok=True)

# Test configuration
TEST_PUZZLE_TYPE = "puzzle_2x2"  # Start with 2x2 for testing
TEST_IMAGE_ID = "0"

# Matching parameters
COMPATIBILITY_THRESHOLD = 10.0  # Lower = more strict matching
TOP_K_MATCHES = 5  # Number of candidate matches per edge

print(f"Testing with: {TEST_PUZZLE_TYPE}/image_{TEST_IMAGE_ID}")

## 3. Load Phase 1 Outputs

Load the processed edge images and enhanced pieces from Phase 1.

In [ ]:
# Load edge images
pieces = load_processed_pieces(PROCESSED_DIR, TEST_PUZZLE_TYPE, TEST_IMAGE_ID)
print(f"Loaded {len(pieces)} pieces")

# Load piece images for visualization
piece_images = load_piece_images(PROCESSED_DIR, TEST_PUZZLE_TYPE, TEST_IMAGE_ID)
print(f"Loaded {len(piece_images)} piece images")

# Display sample pieces
if len(pieces) >= 4:
    fig, axes = plt.subplots(2, 2, figsize=(10, 10))
    axes = axes.flatten()
    
    for idx in range(min(4, len(pieces))):
        piece_id, edge_img = pieces[idx]
        axes[idx].imshow(edge_img, cmap='gray')
        axes[idx].set_title(f"{piece_id}")
        axes[idx].axis('off')
    
    plt.tight_layout()
    plt.suptitle('Sample Edge Images from Phase 1', y=1.02)
    plt.show()

## 4. Extract Edges from All Pieces

Extract the four edges (top, bottom, left, right) from each puzzle piece and compute shape descriptors.

In [ ]:
# Extract grid size from puzzle type
grid_size = int(TEST_PUZZLE_TYPE.split('_')[-1].split('x')[0])
print(f"Grid size: {grid_size}x{grid_size}")

# Extract all edges
all_edges = extract_all_edges_from_puzzle(PROCESSED_DIR, TEST_PUZZLE_TYPE, TEST_IMAGE_ID)

print(f"\nExtracted edges from {len(all_edges)} pieces")
print(f"Expected: {grid_size * grid_size} pieces")

## 5. Visualize Edge Descriptors (Debug)

Examine the shape descriptors computed for individual edges.

In [ ]:
# Visualize descriptors for first piece
if all_edges:
    first_piece_edges = all_edges[0]
    
    print(f"Examining piece with edges: {list(first_piece_edges.keys())}")
    
    # Visualize each edge
    for edge_pos, edge in first_piece_edges.items():
        print(f"\n{edge_pos.upper()} EDGE:")
        visualize_edge_descriptors(edge)

## 6. Find Edge Matches

Compare all edges and find the best matching pairs based on shape similarity.

In [ ]:
# Find matches
print("="*60)
print("FINDING EDGE MATCHES")
print("="*60)

matches = find_edge_matches(
    all_edges, 
    compatibility_threshold=COMPATIBILITY_THRESHOLD,
    top_k=TOP_K_MATCHES
)

print(f"\n{'='*60}")
print(f"Total matches found: {len(matches)}")
print(f"{'='*60}")

# Show top 10 matches
print("\nTop 10 Matches:")
print(f"{'Rank':<6}{'Piece 1':<25}{'Edge 1':<10}{'Piece 2':<25}{'Edge 2':<10}{'Score':<10}")
print("-" * 90)

for idx, (edge1, edge2, score) in enumerate(matches[:10]):
    print(f"{idx+1:<6}{edge1.piece_id:<25}{edge1.edge_position:<10}"
          f"{edge2.piece_id:<25}{edge2.edge_position:<10}{score:.3f}")

## 7. Visualize Edge Matches

Display the top matching edge pairs side-by-side with highlighted edges.

In [ ]:
# Visualize top matches
if matches:
    visualize_edge_matches(matches, piece_images, grid_size, max_matches=min(10, len(matches)))
else:
    print("No matches to visualize")

## 8. Visualize Match Graph

Show all pieces with connections indicating potential matches.

In [ ]:
# Create puzzle graph visualization
if matches:
    visualize_puzzle_graph(
        matches, 
        grid_size, 
        piece_images,
        confidence_threshold=COMPATIBILITY_THRESHOLD,
        figsize=(14, 14)
    )
else:
    print("No matches to visualize")

## 9. Assemble Puzzle

Use the edge matches to assemble the puzzle pieces into their correct positions.

In [ ]:
# Perform assembly
print("="*60)
print("PUZZLE ASSEMBLY")
print("="*60)

if matches:
    solver = PuzzleSolver(grid_size, matches)
    
    # Find corner and border pieces
    corners = solver.find_corner_pieces()
    borders = solver.find_border_pieces()
    
    print(f"\nCorner pieces found: {len(corners)}")
    if corners:
        print(f"  {corners}")
    
    print(f"\nBorder pieces found: {len(borders)}")
    for piece_id, border_dirs in list(borders.items())[:5]:
        print(f"  {piece_id}: borders at {border_dirs}")
    
    # Perform greedy assembly
    print("\nStarting greedy assembly...")
    piece_positions = solver.greedy_assembly()
    
    # Compute quality metrics
    quality = solver.compute_assembly_quality(piece_positions)
    
    print("\n" + "="*60)
    print("ASSEMBLY QUALITY METRICS")
    print("="*60)
    for key, value in quality.items():
        if isinstance(value, float):
            print(f"{key:.<40} {value:.3f}")
        else:
            print(f"{key:.<40} {value}")
    
else:
    print("No matches found - using default grid layout")
    piece_positions = {}
    for idx, piece_edges in enumerate(all_edges):
        piece_id = list(piece_edges.values())[0].piece_id
        row = idx // grid_size
        col = idx % grid_size
        piece_positions[piece_id] = (row, col)

## 10. Visualize Assembly Result

Display the final assembled puzzle with pieces in their computed positions.

In [ ]:
# Visualize assembly
if piece_positions:
    visualize_assembly_result(piece_positions, piece_images, grid_size, figsize=(12, 12))
else:
    print("No assembly to visualize")

## 11. Save Results

Save match reports and assembly results for documentation.

In [ ]:
# Save match report
if matches:
    report_path = Path(OUTPUT_DIR) / f"matches_{TEST_PUZZLE_TYPE}_{TEST_IMAGE_ID}.json"
    save_match_report(matches, report_path, top_n=50)

# Save assembly results
if piece_positions:
    assembly_path = Path(OUTPUT_DIR) / f"assembly_{TEST_PUZZLE_TYPE}_{TEST_IMAGE_ID}.json"
    
    assembly_data = {
        'puzzle_type': TEST_PUZZLE_TYPE,
        'image_id': TEST_IMAGE_ID,
        'grid_size': grid_size,
        'piece_positions': {pid: list(pos) for pid, pos in piece_positions.items()},
        'quality_metrics': quality if 'quality' in locals() else {}
    }
    
    with open(assembly_path, 'w') as f:
        json.dump(assembly_data, f, indent=2)
    
    print(f"Assembly results saved to {assembly_path}")

## 12. Batch Processing (Optional)

Process multiple puzzle images to test robustness.

In [ ]:
def process_multiple_puzzles(puzzle_type, image_ids, processed_dir, output_dir):
    """Process multiple puzzle images and collect statistics."""
    
    results = []
    
    for image_id in image_ids:
        print(f"\n{'='*60}")
        print(f"Processing {puzzle_type}/image_{image_id}")
        print(f"{'='*60}")
        
        try:
            # Extract edges
            all_edges = extract_all_edges_from_puzzle(processed_dir, puzzle_type, image_id)
            
            if not all_edges:
                print(f"  No edges found for image_{image_id}")
                continue
            
            # Find matches
            matches = find_edge_matches(all_edges, compatibility_threshold=10.0, top_k=5)
            
            # Assemble
            grid_size = int(puzzle_type.split('_')[-1].split('x')[0])
            
            if matches:
                solver = PuzzleSolver(grid_size, matches)
                piece_positions = solver.greedy_assembly()
                quality = solver.compute_assembly_quality(piece_positions)
            else:
                quality = {'pieces_placed': 0, 'completion': 0, 'match_accuracy': 0}
            
            results.append({
                'image_id': image_id,
                'num_edges': len(all_edges),
                'num_matches': len(matches),
                **quality
            })
            
            print(f"  Result: {quality.get('completion', 0):.1%} complete, "
                  f"{quality.get('match_accuracy', 0):.1%} accuracy")
            
        except Exception as e:
            print(f"  Error processing image_{image_id}: {e}")
            continue
    
    # Save summary
    summary_path = Path(output_dir) / f"batch_results_{puzzle_type}.json"
    with open(summary_path, 'w') as f:
        json.dump({
            'puzzle_type': puzzle_type,
            'total_processed': len(results),
            'results': results
        }, f, indent=2)
    
    print(f"\nBatch processing complete. Results saved to {summary_path}")
    
    return results

# Example: process first 3 images of 2x2 puzzles
# Uncomment to run:
# batch_results = process_multiple_puzzles(
#     "puzzle_2x2", 
#     ["0", "1", "2"], 
#     PROCESSED_DIR, 
#     OUTPUT_DIR
# )

## 13. Summary and Next Steps

### What We've Accomplished:
1. ✅ Extracted edge contours from puzzle pieces
2. ✅ Computed rotation-invariant shape descriptors (Fourier, curvature, centroid distances)
3. ✅ Implemented edge matching using similarity metrics
4. ✅ Detected border vs. internal edges
5. ✅ Assembled puzzles using greedy algorithm with priority queue
6. ✅ Visualized matches and assembly results
7. ✅ Computed quality metrics for assembly

### Key Insights:
- **Fourier Descriptors** provide rotation invariance for edge matching
- **Curvature signatures** help identify complementary "lock and key" shapes
- **Border detection** using straightness metric improves accuracy
- **Greedy assembly** works well for simple puzzles but may need refinement for complex cases

### Limitations and Future Improvements:
- Current algorithm assumes pieces are not rotated in the captured images
- Matching accuracy decreases with higher complexity puzzles (8x8)
- Could benefit from:
  - More sophisticated distance metrics
  - Backtracking for incorrect placements
  - Global optimization (e.g., simulated annealing)
  - Incorporating piece image content (texture matching)

### For Demo:
- Test on multiple puzzle sizes (2x2, 4x4, 8x8)
- Show clean case (well-separated pieces)
- Show challenging case (noisy background, low contrast)
- Create video demonstration